In [ ]:
from pathlib import Path
import sys

# --- Notebook bootstrap (works from repo root or notebooks/) ---
REPO_ROOT = Path.cwd()
if not (REPO_ROOT / 'mm').exists():
    REPO_ROOT = REPO_ROOT.parent
sys.path.insert(0, str(REPO_ROOT))

DATA_ROOT = REPO_ROOT / 'data'
OUT_ROOT = REPO_ROOT / 'out'

print('REPO_ROOT:', REPO_ROOT)


# Backtest diagnostics notebook

This notebook loads the CSV outputs produced by `PaperExchange` and provides a set of diagnostic views: MTM PnL, inventory, fills, time-to-fill, and basic execution statistics.

**Expected files in `OUT_DIR`:**
- `orders_<SYMBOL>.csv`
- `fills_<SYMBOL>.csv`
- `state_<SYMBOL>.csv`


In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Configure these two fields ---
OUT_DIR = Path(os.environ.get("OUT_DIR", "out_backtest"))
SYMBOL = os.environ.get("SYMBOL", "BTCUSDT")

orders_path = resolve_csv_path(OUT_DIR / f"orders_{SYMBOL}.csv")
fills_path  = resolve_csv_path(OUT_DIR / f"fills_{SYMBOL}.csv")
state_path  = resolve_csv_path(OUT_DIR / f"state_{SYMBOL}.csv")

orders_path, fills_path, state_path

from pathlib import Path

def resolve_csv_path(p: Path) -> Path:
    """Return p if exists; otherwise try p+'.gz'. Accepts either .csv or .csv.gz."""
    if p.exists():
        return p
    gz = Path(str(p) + '.gz')
    if gz.exists():
        return gz
    # also allow replacing suffix if user provided .csv.gz already
    if p.suffix == '.gz' and p.exists():
        return p
    raise FileNotFoundError(f'File not found: {p} (or {gz})')




In [ ]:
orders = pd.read_csv(orders_path)
fills  = pd.read_csv(fills_path)
state  = pd.read_csv(state_path)

# normalize dtypes
for c in ["recv_ms", "active_recv_ms", "expire_recv_ms"]:
    if c in orders.columns:
        orders[c] = pd.to_numeric(orders[c], errors="coerce")
for c in ["recv_ms"]:
    fills[c] = pd.to_numeric(fills[c], errors="coerce")
    state[c] = pd.to_numeric(state[c], errors="coerce")

for c in ["price","qty"]:
    if c in orders.columns:
        orders[c] = pd.to_numeric(orders[c], errors="coerce")
    fills[c] = pd.to_numeric(fills[c], errors="coerce")

if "fee" in fills.columns:
    fills["fee"] = pd.to_numeric(fills["fee"], errors="coerce")

for c in ["inventory","cash","mid","mtm_value"]:
    state[c] = pd.to_numeric(state[c], errors="coerce")

display(orders.head(3), fills.head(3), state.head(3))



## Quick summary

In [ ]:
from mm.backtest.report import summarize, compute_time_to_fill_ms

s = summarize(SYMBOL, orders, fills, state)
s


In [ ]:
print(f"Orders: {s.n_orders:,} | Fills: {s.n_fills:,} | Fill rate: {s.fill_rate:.3f}")
print(f"MTM start: {s.start_mtm:.8f} | MTM end: {s.end_mtm:.8f} | MTM PnL: {s.mtm_pnl:.8f}")
print(f"Realized PnL (avg-cost): {s.realized_pnl:.8f} | Unrealized PnL: {s.unrealized_pnl:.8f}")
print(f"Fees: {s.fees:.8f}")
print(f"End inventory: {s.end_inventory:.8f} | End cash: {s.end_cash:.8f}")


## Time series: mid, inventory, MTM value

In [ ]:
t0 = state["recv_ms"].iloc[0]
t = (state["recv_ms"] - t0) / 1000.0

plt.figure()
plt.plot(t, state["mid"])
plt.xlabel("time (s)")
plt.ylabel("mid")
plt.title(f"{SYMBOL} — Mid price")
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(t, state["inventory"])
plt.xlabel("time (s)")
plt.ylabel("inventory")
plt.title(f"{SYMBOL} — Inventory")
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(t, state["mtm_value"])
plt.xlabel("time (s)")
plt.ylabel("mtm_value")
plt.title(f"{SYMBOL} — MTM value")
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(t, state["mtm_value"] - state["mtm_value"].iloc[0])
plt.xlabel("time (s)")
plt.ylabel("mtm_pnl (relative)")
plt.title(f"{SYMBOL} — MTM PnL (relative to start)")
plt.tight_layout()
plt.show()


## Fills overlayed on mid
Useful to see whether you are systematically adverse-selected (fills clustering at local extrema) or capturing spread.

In [ ]:
if len(fills):
    fills2 = fills.copy()
    fills2["t"] = (fills2["recv_ms"] - t0) / 1000.0

    plt.figure()
    plt.plot(t, state["mid"])
    plt.scatter(fills2["t"], fills2["price"], s=10)
    plt.xlabel("time (s)")
    plt.ylabel("price")
    plt.title(f"{SYMBOL} — Fills over mid")
    plt.tight_layout()
    plt.show()
else:
    print("No fills to plot.")


## Time-to-fill distribution
Computed as `fill.recv_ms - order.active_recv_ms` (requires orders.csv and fills.csv).

In [ ]:
ttf = compute_time_to_fill_ms(orders, fills)
ttf.describe()


In [ ]:
if len(ttf):
    plt.figure()
    plt.hist(ttf.values, bins=60)
    plt.xlabel("time-to-fill (ms)")
    plt.ylabel("count")
    plt.title(f"{SYMBOL} — Time-to-fill histogram")
    plt.tight_layout()
    plt.show()
else:
    print("No time-to-fill values (no fills or missing active_recv_ms).")


## Order lifecycle / TTL sanity checks
Look for large gaps between placement and activation, and ensure expirations align with your config.

In [ ]:
if len(orders):
    orders2 = orders.copy()
    orders2["place_to_active_ms"] = orders2["active_recv_ms"] - orders2["recv_ms"]
    orders2["ttl_ms"] = orders2["expire_recv_ms"] - orders2["active_recv_ms"]
    display(orders2[["place_to_active_ms","ttl_ms"]].describe(percentiles=[0.5,0.9,0.99]))
else:
    print("No orders.")


## Inventory vs mid moves (simple adverse selection proxy)
If inventory increases when mid is falling (or decreases when mid is rising), that can indicate adverse selection.

This is a crude view but helpful early on.

In [ ]:
# align to same index for correlation
mid_ret = state["mid"].diff()
inv_chg = state["inventory"].diff()

df = pd.DataFrame({"mid_ret": mid_ret, "inv_chg": inv_chg}).dropna()
df.corr()


In [ ]:
plt.figure()
plt.scatter(df["mid_ret"], df["inv_chg"], s=8)
plt.xlabel("Δ mid")
plt.ylabel("Δ inventory")
plt.title(f"{SYMBOL} — Inventory change vs mid change")
plt.tight_layout()
plt.show()
